In [1]:
import graph_tool.all as gt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import itertools

## Data Import and Set Up

The starting point for this project is an existing dataset of non-owner occupied parcels in the City of Atlanta. It joins together two datasets from the county tax assessor's offices in Fulton and DeKalb counties and comes from a previous project that disambiguated property ownership using a number of programs and manually checking techniques. For the purposes of this project, it's important to note that the `OwnerName` and `OwnerAddress` columns have already been lightly processed to match together small typos and misspellings.

The original dataset is imported in its entirety then immediating a "thinner" dataframe is created with the four columns needed for this analysis

In [2]:
df = pd.read_csv("../Data/AtlCity_NotOwnerParcels.csv")

In [3]:
df.columns

Index(['RowID', 'County', 'ParcelID', 'SiteAddress', 'SiteNumStr', 'SiteNum',
       'SiteStreet', 'SiteZip', 'SiteState', 'OwnerName', 'OwnerAddress',
       'OwnerNumStr', 'ClstName', 'ClstAddress', 'N_OName', 'N_OAddress',
       'RealOwner', 'OwnerCat', 'ZoneCode', 'LUCode', 'TotAppr', 'LandAppr',
       'BldgAppr', 'LotSize', 'ParcelLong', 'ParcelLat'],
      dtype='object')

In [4]:
sdf = df[['SiteAddress', 'OwnerName', 'OwnerAddress', 'ClstName', 'ClstAddress', 'N_OName', 'N_OAddress',
       'RealOwner']].copy()

In [5]:
sdf["Pid"] = sdf.index + 1 # Creates a new ID so parcels can be tied back to addresses throughout the project

In [6]:
sdf

,SiteAddress,OwnerName,OwnerAddress,ClstName,ClstAddress,N_OName,N_OAddress,RealOwner,Pid
0,"1001 Hilburn Drive Atlanta, GA 30316",2018 2 IH BORROWER LP,"1717 MAIN ST STE 2000 DALLAS, TX 75201",2018-4 IH BORROWER LP,"1717 MAIN ST STE 2000, DALLAS, TX, 75201-4657",2901,5621,Invitation,1
1,"914 Bouldercrest Drive Atlanta, GA 30316",2018 2 IH BORROWER LP,"1717 MAIN ST STE 2000 DALLAS, TX 75201",2018-4 IH BORROWER LP,"1717 MAIN ST STE 2000, DALLAS, TX, 75201-4657",2901,5621,Invitation,2
2,"2170 Memorial Drive Atlanta, GA 30317",2017-2 IH BORROWER LP,"1717 MAIN ST STE 2000 DALLAS, TX 75201",2018-4 IH BORROWER LP,"1717 MAIN ST STE 2000, DALLAS, TX, 75201-4657",2901,5621,Invitation,3
3,"316 Patterson Avenue Atlanta, GA 30316",2013-1 IH BORROWER LP,"1717 MAIN ST STE 2000 DALLAS, TX 75201",2018-4 IH BORROWER LP,"1717 MAIN ST STE 2000, DALLAS, TX, 75201-4657",2901,5621,Invitation,4
4,"55 Wyman Street SE Atlanta, GA 30317",2017-2 IH BORROWER LP,"1717 MAIN ST STE 2000 DALLAS, TX 75201",2018-4 IH BORROWER LP,"1717 MAIN ST STE 2000, DALLAS, TX, 75201-4657",2901,5621,Invitation,5
...,...,...,...,...,...,...,...,...,...
73783,"270 SEVENTEENTH ST UNIT 2903, GA",CLARK MELINDA ANN,"270 17TH ST NW UNIT 2903, ATLANTA GA 30363",CLARK MELINDA ANN,"270 17TH ST NW UNIT 2903, ATLANTA GA 30363",1,1,,73784
73784,"229 PEACHTREE HILLS AVE UNIT 5303, GA",MIXSON HOMER LAMAR,"2500 W WESLEY RD, ATLANTA GA 30327",MIXSON HOMER LAMAR,"2500 W WESLEY RD, ATLANTA GA 30327",1,1,,73785
73785,"229 PEACHTREE HILLS AVE UNIT 334, GA",BROOKWOOD PRIDE LLC,"229 PEACHTREE HILLS AVE UNIT 334, ATLANTA GA 3...",BROOKWOOD PRIDE LLC,"229 PEACHTREE HILLS AVE UNIT 334, ATLANTA GA 3...",1,1,,73786
73786,"270 SEVENTEENTH ST UNIT 2410, GA",SHERWOOD DELESHIA,"270 17TH ST NW UNIT 2410, ATLANTA GA 30363",SHERWOOD DELESHIA,"270 17TH ST NW UNIT 2410, ATLANTA GA 30363",1,1,,73787


In [7]:
# Saves the parcel ID dataframe to a CSV
sdf.to_csv("../Data/ParcelIDed.csv")

## Converting the dataframes into a network ready format

The goal of this project is to use network analysis to identify clusters of addresses and owner names that might indicate common ownership that can't be seen by simply grouping observation together based on that information. To do this, the dataframe of parcel records needs to be converted into some form of node and edge list.

Easiest thing to do in `graph-tools` is going to be having vertex pairs

### Networking the parcel addresses by shared owner name/address

Owner Address

In [8]:
# Group the parcel dataframe by OwnerAddress and store all of the ids that share the address
gdf = sdf.groupby('OwnerAddress')['Pid'].apply(list).reset_index()

In [9]:
# Count the number of parcels in each group to then sort the list and see the top addresses
def groupcount(row):
    return len(row['Pid'])

gdf["GroupSize"] = gdf.apply(groupcount, axis = 1)

In [10]:
gdf.sort_values('GroupSize', ascending=False).head(10)

,OwnerAddress,Pid,GroupSize
35943,"68 MITCHELL ST SW STE 1350, ATLANTA GA 30303","[1696, 1697, 1700, 1701, 1702, 1703, 1704, 170...",893
8,.,"[582, 583, 584, 585, 586, 587, 588, 589, 590, ...",848
18216,"270 WASHINGTON ST SW, ATLANTA GA 30334","[4208, 4210, 4211, 4212, 4213, 4214, 4215, 421...",417
6731,"141 PRYOR ST SW SUITE 7000, ATLANTA GA 30303","[3706, 3707, 3708, 3709, 3710, 3711, 3712, 371...",276
33874,"591 W PUTNAM AVE, GREENWICH CT 06830","[3080, 3082, 3673, 5018, 5019, 5021, 5022, 502...",231
38822,"824 MEMORIAL DR SE, ATLANTA GA 30316","[5464, 5466, 5468, 5470, 5472, 5473, 5474, 547...",217
16135,"2424 PIEDMONT RD NE ATLANTA, GA 30324","[3168, 3169, 3170, 3171, 3172, 3173, 3174, 317...",215
35952,"68 MITCHELL ST SW STE 1350, ATLANTA GA 30303-3534","[1684, 1685, 1686, 1687, 1688, 1689, 1690, 169...",210
16046,"241 RALPH MCGILL BLVD NE, ATLANTA GA 30308-3374","[4820, 4821, 4822, 4823, 4824, 4825, 4826, 482...",190
24028,"3505 KOGER BLVD #400, DULUTH GA 30096","[135, 139, 141, 142, 143, 144, 147, 149, 151, ...",185


Now, what I want to do is unpack the Pid column as two lists of sources and ends that represent all of the possible combinations within those Pid lists. Corresponding, I want a equally sized list that records the Address of the cluster, so I can name the edge later. Then a fourth list that records the type of the edge (i.e., address or name)

Those four lists are then join together to create a zipped list of tuples that can be fed into graph-tools

In [11]:
# Heavily relying on things not moving around and the list maintaining the correct order to match sources and ends
sources = []
ends = []
names = []

for index, row in gdf.iterrows():
    name = row['OwnerAddress'] # For every,save the OwnerAddress value to a variable
    for x in itertools.combinations(row['Pid'], 2): # Iterates through all combinations of points in the Pid list
        sources.append(str(x[0])) # Add the first point in the pair to the sources list
        ends.append(str(x[1])) # Add the second point to the ends list
        # Source and end ids were saved as strings in order for graph-tools to create valid hashed node ids for them
        names.append(name) # For every combination, save the name value into the names list

types = ["Address"]*len(sources) # Make an identical length list with the repeated value "Address" to denote the type of node

In [12]:
# Checking the length of the lists to make sure they are all the same
print(len(sources))
print(len(ends))
print(len(names))
print(len(types))

1348062
1348062
1348062
1348062


In [13]:
# Zipping the lists into a list of tuples
AddressEdges = list(zip(sources, ends, names, types))

In [14]:
# Sample of tuple list
AddressEdges[1:10]

[('15826', '21113', '#2 CAPITAL SQUARE, ATLANTA, GA, 30334', 'Address'),
 ('15827', '21113', '#2 CAPITAL SQUARE, ATLANTA, GA, 30334', 'Address'),
 ('582', '583', '.', 'Address'),
 ('582', '584', '.', 'Address'),
 ('582', '585', '.', 'Address'),
 ('582', '586', '.', 'Address'),
 ('582', '587', '.', 'Address'),
 ('582', '588', '.', 'Address'),
 ('582', '589', '.', 'Address')]

Success! That list should be able to be passed to graph-tools in order to creates vertices and edges

In [15]:
# Creates Address edge network with hashed point names and extra edge properties
AdrG = gt.Graph(AddressEdges, eprops = [('name', 'string'), ('type', 'string')], hashed = True, directed = False)

In [16]:
# Check the number of vertices and edges
print("Number of vertices: {}".format(len(list(AdrG.vertices()))))
print("Number of edges: {}".format(len(list(AdrG.edges()))))

Number of vertices: 36494
Number of edges: 1348062


In [17]:
# Save the output in the graph-tools format
AdrG.save("../Data/address_graph.gt.gz")

Owner Name

In [18]:
# Group the parcel dataframe by OwnerAddress and store all of the ids that share the address
g2df = sdf.groupby('OwnerName')['Pid'].apply(list).reset_index()

In [19]:
# Count the number of parcels in each group to then sort the list and see the top addresses
def groupcount(row):
    return len(row['Pid'])

g2df["GroupSize"] = g2df.apply(groupcount, axis = 1)

In [20]:
g2df.sort_values('GroupSize', ascending=False).head(10)

,OwnerName,Pid,GroupSize
9486,CITY OF ATLANTA,"[1596, 1597, 1598, 1600, 1601, 1602, 1604, 160...",1346
0,,"[582, 583, 584, 585, 586, 587, 588, 589, 590, ...",817
27269,MARTA,"[3168, 3169, 3170, 3171, 3172, 3173, 3174, 317...",474
18237,HABITAT FOR HUMANITY IN ATLANTA INC,"[5464, 5465, 5466, 5467, 5469, 5470, 5471, 547...",325
15815,FULTON COUNTY,"[3697, 3698, 3701, 3702, 3703, 3704, 3705, 370...",323
35100,REGENTS UNIV SYSTEM OF GA,"[4208, 4209, 4210, 4211, 4212, 4213, 4214, 421...",257
15894,FYR SFR BORROWER LLC,"[129, 130, 134, 135, 139, 141, 142, 143, 144, ...",207
16529,GEORGIA POWER COMPANY TAX DEPT BIN 10120,"[4820, 4821, 4822, 4823, 4824, 4825, 4826, 482...",185
39697,STAR 2021 SFR1 BORROWER LP,"[5018, 5019, 5020, 5021, 5022, 5023, 5024, 502...",180
12065,DEPARTMENT OF TRANSPORTATION,"[5859, 6598, 6599, 6600, 6601, 6602, 6604, 660...",179


In [21]:
# Converts the OwnerName groups to edge lists, using the same process as above
sources = []
ends = []
names = []

for index, row in g2df.iterrows():
    name = row['OwnerName']
    for x in itertools.combinations(row['Pid'], 2):
        sources.append(str(x[0]))
        ends.append(str(x[1]))
        names.append(name)

types = ["Name"]*len(sources)

NameEdges = list(zip(sources, ends, names, types))

# Creates the Name edge network in the same format as the Address Network
NamG = gt.Graph(NameEdges, eprops = [('name', 'string'), ('type', 'string')], hashed = True, directed = False)

In [22]:
print("Number of vertices: {}".format(len(list(NamG.vertices()))))
print("Number of edges: {}".format(len(list(NamG.edges()))))

Number of vertices: 34368
Number of edges: 1879701


In [23]:
NamG.save("../Data/name_graph.gt.gz")

Joining them into one network

In [24]:
# Easiest way to create a network that uses the same nodes but combines both types of edges
# is to feed in one expended list of edges
allEdges = AddressEdges + NameEdges
len(allEdges) # Is the sum of the two previous edge lists

3227763

In [25]:
# Creates a network that contains both Address and Name edges using the same format of the two previous networks
JG = gt.Graph(allEdges, eprops = [('name', 'string'), ('type', 'string')], hashed = True, directed = False)

In [26]:
# The number of nodes represents the merged nodes in the two previous networks,
# the number of edges is the simple sum
print("Number of vertices: {}".format(len(list(JG.vertices()))))
print("Number of edges: {}".format(len(list(JG.edges()))))

Number of vertices: 41396
Number of edges: 3227763


In [27]:
# Saves the output to be imported into the subsequent script
JG.save("../Data/joint_graph.gt.gz")

Decompisition of the network graph into list component IDs for parcels occurs in the *Parcel Component Analysis* script.